# Use `seaduck.OceInterp` with ECCO

`seaduck` Lagrangian particle demonstration. This version uses a reduced version of the ECCO MITgcm velocity field data.

authors: Wenrui Jiang, Tom Haine Feb '23

In [ ]:
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

import seaduck as sd

mpl.rcParams["figure.dpi"] = 300

## Loading Dataset

The ECCO MITgcm run is a low-resolution global state estimate. An artifact of note is that this dataset has complex grid topology, which means there is a face (also called tile) dimension in the dataset.

A built-in function in `seaduck.utils` can help access the snippet of ECCO that this example is based on.

In [ ]:
ecco = sd.utils.get_dataset("ecco")
ecco

```{admonition} Access full ECCO dataset
The ECCO dataset is publicly available on SciServer. The simulation output could opened using the OceanSpy package using the [`from_catalog`](https://oceanspy.readthedocs.io/en/latest/generated/oceanspy.open_oceandataset.from_catalog.html#oceanspy.open_oceandataset.from_catalog) method.

Choose between the monthly-mean data ('ECCO')

`ecco = ospy.open_oceandataset.from_catalog("ECCO")._ds`

or the daily-mean data ('daily_ecco').

`ecco = ospy.open_oceandataset.from_catalog('daily_ecco')._ds`

Click [here](https://dev-poseidon-ocean.pantheonsite.io/products/datasets/) for a full list of the dataset hosted.
```

## Experiment setup

Specify the parameters for the particles (number, positions, start time).

In [ ]:
# Define the extend of the box
west = -90.0
east = 0.0
south = 23.0
north = 67.0
shallow = -10.0
deep = -10.0

time = "1992-02-15"

Nlon = 80  # How many along longitudinal direction?
Nlat = 80  # How many along latitudinal direction?
Ndep = 1  # How many along vertical direction?

x, y, z, t = sd.utils.easy_3d_cube(
    (east, west, Nlon),
    (south, north, Nlat),
    (shallow, deep, Ndep),
    time,
    print_total_number=True,
)

Plot the particle positions

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.plot(x, y, "ro", markersize=0.5)
ax.coastlines()
# ax.set_xlim([-25, 0])
# ax.set_ylim([50, 70])
ax.set_title("Interpolation position")
plt.show()

## Let's explore `seaduck.OceInterp`

This is the most high-level function of the package. Yes, it is very easy to use.

In [ ]:
help(sd.OceInterp)

### Interpolate these ECCO fields at Eulerian positions.

In [ ]:
[s, (u, v), eta, mask] = sd.OceInterp(
    ecco, ["SALT", ("UVELMASS", "VVELMASS"), "ETAN", "maskC"], x, y, z, t
)

Plot the interpolated salinity, $u$, $\eta$ field.

In [ ]:
unit = ["psu", "m/s", "m"]
name = ["Salinity", "Zonal Velocity", "Sea Surface Height"]
for i, var in enumerate([s, u, eta]):
    ax = plt.subplot(1, 3, 1 + i, projection=ccrs.PlateCarree())
    c = ax.scatter(x, y, c=var, s=0.5)
    ax.coastlines()
    ax.set_xlim([west, east])
    ax.set_ylim([south, north])
    plt.colorbar(c, location="bottom", label=unit[i], pad=0.03)
    ax.set_title(name[i])
plt.tight_layout()
plt.show()

The salinity and the sea surface height variable here are not model output but randomly generated noise and there are values on land as well. However, the package respects the mask provided by the model, so even though there is value, NaNs are returned.

This is not the case for velocity. The mask for the staggered velocity field is not provided by the model, so the actual value (zero here) is returned.

### Now compute Lagrangian trajectories for these particles.

First, define the `start_time` and `end_time`. Here the particles are integrated backwards in time.

In [ ]:
start_time = "1992-01-17"
end_time = "1992-03-12"

t_bnds = np.array(
    [
        sd.utils.convert_time(start_time),
        sd.utils.convert_time(end_time),
    ]
)

### Perform the particle trajectory simulation.

To switch between Lagrangian and Eulerian, you only need to change the `lagrangian` keyword argument.

The following code block simulate the trajectory and record the salinity along the particle trajectories as well as their (lat,lon) positions.

In [ ]:
stops, [s, raw, lat, lon] = sd.OceInterp(
    ecco,
    ["SALT", "__particle.raw", "__particle.lat", "__particle.lon"],
    x,
    y,
    z,
    t_bnds,
    lagrangian=True,
    return_pt_time=True,
)

There are 4 output times. See also the diagnostic output from running the integration.

In [ ]:
len(stops)

The `raw` output is a list of `seaduck.Position` objects which stores, of course, the position of the particle at specific times.

In [ ]:
raw

### Plot the interpolated salinity field on the final particle positions.

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.scatter(lon[-1], lat[-1], c=s[-1], s=1)
ax.coastlines()
ax.set_xlim([-70, 0])
ax.set_ylim([30, 70])
plt.title("salinity map")
plt.show()

## Calculate derivatives

### Kernel object

The kernel object defines which neighboring points are used for the interpolation, and also what kind of operation is conducted. The default is interpolation. However, you can also use this class to calculate derivatives.

In [ ]:
KnW = sd.kernel_weight.KnW
help(KnW)

Let's define derivative kernels for $\partial / \partial z$, $\partial^2 / \partial x^2$, and $\partial / \partial t$ as an example

In [ ]:
default = KnW()
dz_kernel = KnW(vkernel="dz")
dx2_kernel = KnW(hkernel="dx", h_order=2, inheritance=None, tkernel="linear")
dt_kernel = KnW(tkernel="dt")

Apply the kernels to the ECCO fields:

In [ ]:
[dsdz, (d2udx2, dvdt)] = sd.OceInterp(
    ecco,
    {"SALT": dz_kernel, ("UVELMASS", "VVELMASS"): (dx2_kernel, dt_kernel)},
    x,
    y,
    z,
    t,
)

### Plot one of the differentiated fields on the initial particle positions.

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
c = ax.scatter(x, y, c=d2udx2)
ax.coastlines()
ax.set_xlim([-25, 0])
ax.set_ylim([50, 70])
plt.colorbar(c, location="bottom", label="m/s per grid scale squared")
plt.title("Second Derivative of the Zonal Velocity")
plt.show()